In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import re
import os
import glob
import datetime
from datetime import datetime as dat
import jdatetime
from jdatetime import datetime as jdt
from numpy import random as nprnd
import persiantools
from persiantools import characters
import networkx as nx
from pyvis.network import Network
from networkx import NetworkXError
from networkx import bipartite
import networkx.algorithms.connectivity as nxcon
from networkx.algorithms import approximation as nxp
import networkx.algorithms.community as nxcom
import itertools
import math
import time
import pyodbc
from difflib import SequenceMatcher
import html

# Function

In [ ]:
def excel_to_csv(path,dirr):
    extention='xlsx'
    os.chdir(path)
    for file in os.listdir(path):
        file_xlsx= pd.read_ecxel(file)
        file_xlsx.to_csv(dirr+"\\"+file.split('.')[0]+".csv",index=False, encoding='utf-8-sig')

In [ ]:
def csv_to_xlsx(path,dirr):
    extention='csv'
    os.chdir(path)
    for file in os.listdir(path):
        file_xlsx= pd.read_csv(file)
        file_xlsx.to_excel(dirr+"\\"+file.split('.')[0]+".xlsx",index=False)

In [ ]:
def read_merge_output(path,dirr=None):
    input_1=input('select the type of file: (csv/xlsx) ')
    all_df=[]
    if input_1=='xlsx':
        input_2=input('insert the number of columns: ')
        input_2= int(input_2)
        for file in os.listdir(path):
            df=pd.read_xlsx(path+'\\'+file)
            file_name= file.split('.')[0]
            column_list= df.columns.tolist()
            len_df= len(column_list)
            if len_df== input_2:
                all_df.append(df)
            else:
                print(f'the number of columns of file {file_name} is {len_df}')
                input_3= input('remove column or add column: (add/remove) ')
                if input_3 == 'add':
                    input_4= input('name of column: ')
                    df[input_4]=None
                    all_df.append(df)
                if input_3=='remove':
                    input_5= input('name of removed column: ')
                    df.drop(columns=input_5)
                    all_df.append(df)
    if input_1=='csv':
        input_2=input('insert the number of columns: ')
        input_2= int(input_2)
        for file in os.listdir(path):
            df=pd.read_csv(path+'\\'+file)
            file_name= file.split('.')[0]
            column_list= df.columns.tolist()
            len_df= len(column_list)
            if len_df== input_2:
                all_df.append(df)
            else:
                print(f'the number of columns of file {file_name} is {len_df}')
                input_3= input('remove column or add column: (add/remove) ')
                if input_3 == 'add':
                    input_4= input('name of column: ')
                    df[input_4]=None
                    all_df.append(df)
                if input_3=='remove':
                    input_5= input('name of removed column: ')
                    df.drop(columns=input_5)
                    all_df.append(df)
    input_6=input('do you want output: (y/n) ')
    if input_6=='y':
        df_raw= pd.concat(all_df,ignore_index=True).reset_index(drop=True)
        input_7=input('file type: (xlsx\csv) ')
        if input_7== 'xlsx':
            input_8= ('file name: ')
            df_raw.to_excel(dirr+'\\'+input_8+'.xlsx',index=False)
        if input_7== 'csv':
            input_8= ('file name: ')
            df_raw.to_csv(dirr+'\\'+input_8+'.csv',index=False)
    if input_6=='n':
        df_raw= pd.concat(all_df,ignore_index=True).reset_index(drop=True)
    return df_raw

In [ ]:
def change_column_name(df):
    columns= df.columns.tolist()
    print(f'column name is: {columns}')
    input_1= input('do you want to change header: (n\y)')
    if input_1=='y':
        for i in columns:
            input_2= input(f'insert new name for {i}: ')
            df.rename(columns={i:input_2},inplace=True)

In [ ]:
def column_clustering(df):
    columns= df.columns.tolist()
    print(f'column name is: {columns}')
    string_columns=[]
    code_columns=[]
    price_columns=[]
    input_1=None
    while input_1!='finish':
        input_1= input('choose type of clusters: (string\price\code) ')
        if input_1=='string':
            input_2=None
            while input_2!='finish':
                input_2=input('choose string column')
                string_columns.append(input_2)
            if 'finish' in string_columns:
                string_columns.remove('finish')
        if input_1=='code':
            input_2=None
            while input_2!='finish':
                input_2=input('choose code column')
                code_columns.append(input_2)
            if 'finish' in code_columns:
                code_columns.remove('finish')
        if input_1=='price':
            input_2=None
            while input_2!='finish':
                input_2=input('choose price column')
                price_columns.append(input_2)
            if 'finish' in price_columns:
                price_columns.remove('finish')

    return string_columns,code_columns,price_columns

In [ ]:
def code_clean(df, code_columns):
    for i in code_columns:
        try:
            df[i]= df[i].replace(['nan',np.nan,'NaN'],1000000000001234)
            df[i]=df[i].replace(r'-','',regex=True)
            df[i]=df[i].apply(np.int64)
            df[i]= df[i].astype(str)
            df[i]= df[i].replace(r'1000000000001234',np.nan, regex=True)
        except:
            print(f'error is in {i}')
    return df

In [ ]:
def price_clean (df, price_columns):
    input_1= input('type of price: (float/int)')
    if input_1=='int':
        for i in code_columns:
            try:
                df[i]= df[i].apply(lambda x: x.astype(str))
                df[i]=df[i].apply(lambda x: x(np.int64))
            except:
                print(f'error is in {i}')
    if input_1=='float':
        for i in code_columns:
            try:
                df[i]= df[i].apply(lambda x: x.astype(float))
            except:
                print(f'error is in {i}')
    return df


In [ ]:
def string_clean(df,string_columns):
    for i in string_columns:
        try:
            df[i]= df[i].apply(lambda x: x.astype(str))
            df[i]=df[i].str.lower()
            df[i]=df[i].str.strip()
            df[i]=df[i].replace('  ',' ')
            df[i]=df[i].replace('(','-')
            df[i]=df[i].replace(')','')
            df[i]=df[i].replace('_',' ')
            df[i]=df[i].apply(lambda x: characters.ar_to_fa(x))
            df[i]=df[i].replace(regex=r'\u200c',value=' ')
            df[i]=df[i].str.strip(' ')
            df[i]=df[i].replace(regex=['مجتمع','سهامی خاص','شرکت','سهامی عام'],value='')
        except:
            print(f'error is in {i}')
    return df

In [ ]:
def P_E (df):
    def unique_value(df,column):
        x= df[column].drop_duplicates()
        return x
    columns= df.columns.tolist()
    print(f'list of columns: {columns}')
    columns_list=[]
    input_1=None
    while input_1 != 'finish':
        input_1= input('select column: ')
        columns_list.append(input_1)
    if 'finish' in columns_list:
        columns_list.remove('finish')
    for j in columns_list:
        u = unique_value(df,[j])
        input_2= input(f'{j} in your dataframe has {len(u)} unique value; do you want to replace: (N/Y)')
        if input_2=='n':
            continue
        else:
            dict_replace={}
            c=1
            for i in u.iloc[:,0]:
                input_3= input(f'{c} what is replace of {i}')
                dict_replace[i]= input_3
                if input_3 =='break':
                    break
                else:
                    c+=1
        input_4= input('do you want creat new column?')
        if input_4 == 'break':
            break
        else:
            if input_4=='y':
                name=''
                input_5= input('name of new column: ')
                df[f'{input_5}']= df[j].replace(dict_replace)
            else:
                df[j]= df[j].replace(dict_replace)


In [ ]:
def words_clean(df,columns_list:list):
    for j in columns_list:
        v= df[j].value_counts()
        input_1= input(f'{j} has: {v}/n do wou want to clean: (n/y)')
        if input_1=='n':
            continue
        else:
            list_replace=[]
            input_2=None
            while input_2 != 'finish':
                input_2= input ('what is the words to remove: ')
                list_replace.append(input_2)
            if 'finish' in list_replace:
                list_replace.remove('finish')
            input_3= input('do you want to creat new column: ')
            if input_3 =='break':
                break
            else:
                if input_3=='y':
                    name=''
                    input_4= input('new column name: ')
                    df[f'{input_4}']= df[j].replace(regex=list_replace,value='')
                else:
                    df[j]= df[j].replace(regex=list_replace,value='')

In [ ]:
def remove_extra_words (pd_series):
    pd_series= pd_series.astype(str)
    pd_series= pd_series.replace(r'سهامی عام','',regex=True)
    pd_series= pd_series.replace(r'شرکت','',regex=True)
    pd_series= pd_series.replace(r'مجتمع','',regex=True)
    pd_series= pd_series.replace(r'سهامی خاص','',regex=True)
    pd_series= pd_series.str.strip()
    pd_series= pd_series.str.strip(' ')
    return pd_series

In [ ]:
def remove_extra_space (pd_series):
    pd_series=pd_series.astype(str)
    pd_series=pd_series.str.strip(' ')
    return pd_series

In [ ]:
def remove_space(pd_series):
    pd_series= pd_series.astype(str)
    pd_series.replace(r' ','',regex=True)
    return pd_series

In [ ]:
def date_timestamp(pd_series):
    pd_series=pd_series.astype(str)
    date = pd_series.apply (lambda x: str(x).split(' ')[0])
    return date

In [ ]:
def date_corrector(pd_series):
    x= pd_series.astype(str)
    year= x.apply(lambda x: str(x)[0:4])
    month= x.apply(lambda x: str(x)[4:6])
    day= x.apply(lambda x: str(x)[6:])
    date = year +'-'+month+'-'+day
    pd_series= date
    pd_series= pd_series.replace(regex='<NA>--', value= np.nan)
    return pd_series

In [ ]:
def finding_value_str(df):
    y = df.columns.tolist()
    print('Note: it takes case sensitive values.')
    keyword= input('keyword for search')
    dict_name={}
    for k in y:
        try:
            l= df[df[k].str.contains(keyword)]
            dict_name[k]= l.index.tolist()
        except:
            continue
    print(dict_name)

In [ ]:
def view(df):
    css = """<style>
    table {border-collapse: collapse; border: 3px solid #eee; }
    table tr th: first-child {background-color: #eee; color: #000; }
    table thead th {background-color: #eee; color:#000 }
    tr, th, td {border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px: font-family: monospace; font-size: 10px }<style>
    """
    s= '<script type = "text/Javascript">'
    s+= 'var win = window.open("", "Title","toolbar=no", location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400+", left="+(screen.width-840"));'
    s+= (
        "win.document.body.innerHTML = '"
        + (df.to_html())+ css).replace("\n","\\")
        + "';"
    )
    s+= "</script>"
    return HTML(s+css)


In [ ]:
def get_snapshot (G, dt):
    '''Convert date to integer timestamp'''
    # dt= datetime.datetime.strptime(date, '%Y-%m-%d)
    timestamp= time.mktime(dt.timetuple())

    # Find edges that exsisted during timestamp
    snapshot_edge=[]
    for e in G.edges:
        if G.edges[e]['begin']<= timestamp and G.edges[e]['end'] >= timestamp:
            snapshot_edges.append(e)
    return nx.Graph(G.edge_subgraph(snapshot_edges))

In [ ]:
def similar_str (df):
    list_similar=[]
    name_list=[]
    company_name= list(dict.fromkeys(df['company_name']))
    person_name= list(dict.fromkeys(df['person_name']))
    for i, j in itertools.product(name_list, list(dict.fromkeys(df['person_anme']))):
        s= SequenceMatcher(None, j, i).ratio()
        if s>= 0.6 and s<1:
            list_index= df.loc[df['person_name']==j].index.tolist()
            sim= {i : list_index}
            list_similar.append(sim)
    return list_similar

In [ ]:
def similar_str_map (df,i,j):
    list_similar=[]
    s= SequenceMatcher(None, j, i).ratio()
    if s>= 0.6 and s<1:
        list_index= df.loc[df['person_name']==j].index.tolist()
        sim= {i : list_index}
        list_similar.append(sim)
    return list_similar

In [ ]:
def checking_words(df):
    '''this function find differences in dictation or code of person name with natioanl code'''
    def check(df):
        national_id_list= list(dict.fromkeys(df['person_national_code']))
        if 'nan' in national_id_list:
            national_id_list.remove('nan')
        list_for_check=[]
        person_with_nan= []
        for i in national_id_list:
            list_index= df.loc[df['person_national_code']==i].index.tolist()
            sub_df= df.loc[list_index]
            list_person_name= list(dict.fromkeys(sub_df['person_name']))
            if 'nan' in list_person_name:
                print(f'there is nan in {i} person name')
                person_with_nan.append(i)
            if len(list_person_name)>1:
                list_for_check.append({i : list_person_name})
        return list_for_check
    list_for_check=['0']
    c=1
    while len(list_for_check)!=0:
        start_time= datetime.datetime.now()
        print('------------------------------------------------------------------------')
        print(f'step_{str(c)}')
        list_for_check= check(df)
        if len(list_for_check) > 0 :
            print('there is problem in word dictation or code')
            print(list_for_check)
            for i in list_for_check:
                for key,value in i.items():
                    list_index_1= df.loc[df['person_national_code']==key].index.tolist()
                    sub_df_1= df.loc[list_index_1]
                    list_person_1= list(dict.fromkeys(sub_df_1['person_name']))
                    for j in list_person_1:
                        input_3= input(f'for {j} : dictation or code or nothing: ')
                        if input_3== 'dict' or input_3=='یهزف':
                            input_4= input(f'enter new dictation for {i} :')
                            df['person_name'] = df['person_name'].replace([j], input_4)
                        if input_3=='code' or input_3=='زخیث':
                            input_4= input(f'enter new code for {j} :')
                            df['person_national_code']= df['person_national_code'].mask(df['person_name']==j, input_4)
                        if input_3=='nothing' or input_3=='دخفاهدل':
                            continue
            c= c+1
        end_time= datetime.datetime.now()
        print('Duration: {}'.format(end_time - start_time))
    print('there is no problem in word dictation or code')
    return df

In [ ]:
def checking_company_person(df, national_company_series,national_person_series,company_name_series,person_name_series):
    """ checking dictation and code of company name and person name"""
    def ckeck(df, national_company_series,national_person_series,company_name_series,person_name_series):
        national_id_list= list(dict.fromkeys(df[national_company_series]))
        if 'nan' in national_id_list:
            national_id_list.remove('nan')
        list_for_check=[]
        person_with_nan=[]
        company_with_nan=[]
        for i in national_id_list:
            list_index_company= df.loc[df[national_company_series]==i].index.tolist()
            list_index_person= df.loc[df[national_person_series]==i].index.tolist()
            sub_df_person= df.loc[list_index_person]
            sub_df_company= df.loc[list_index_company]
            list_person_name= list(dict.fromkeys(sub_df_pesron[person_name_series]))
            list_company_name= list(dict.fromkeys(sub_df_company[company_name_series]))
            if 'nan' in list_person_name:
                print(f'there is nan in {i} {person_name_series}')
                list_person_name.remove('nan')
                person_with_nan.append(i)
            if 'nan' in list_company_name:
                print(f'there is nan in {i} {company_name_series}')
                list_company_name.remove('nan')
                company_with_nan.append(i)
            if len(list_person_name)>1 or len (list_company_name)>1:
                for j in list_company_name:
                    list_person_name.append(j)
                list_person_name= list(dict.fromkeys(list_person_name))
                list_for_check.append({i : list_person_name})
        return list_for_check
    list_for_check=['0']
    c = 1
    while len(list_for_check)!= 0:
        start_time= datetime.datetime.now()
        print('----------------------------------------------------------------------------')
        print(f'step_{str(c)}')
        list_for_check= ckeck(df, national_company_series,national_person_series,company_name_series,person_name_series)
        if len(list_for_check)>0:
            print('there is problem in word dictation or code')
            print(list_for_check)
            for i in list_for_check:
                for key,value in i.items():
                    list_index_company= df.loc[df[national_company_series]==key].index.tolist()
                    list_index_person= df.loc[df[national_person_series]== key].index.tolist()
                    sub_df_company= df.loc[list_index_company]
                    suc_df_person= df.loc[list_index_person]
                    list_person_name= list(dict.fromkeys(suc_df_person[person_name_series]))
                    list_company_name= list(dict.fromkeys(sub_df_company[company_name_series]))
                    for x in list_company_name:
                        list_person_name.append(x)
                    list_person_name= list(dict.fromkeys(list_person_name))
                    for j in list_person_name:
                        input_3= input(f'for {j} : dictation or code or nathing: ')
                        if input_3== 'dict' or input_3== 'یهزف':
                            input_4= input(f'enter new dictation for {j}')
                            df[company_name_series] = df[company_name_series].replace([j], input_4)
                            df[person_name_series] = df[person_name_series].replace([j], input_4)
                        if input_3== 'code' or input_3== 'زخیث':
                            input_4 = input(f'enter new code for {j}')
                            df[national_company_series] = df[national_company_series].mask(df[company_name_series]==j, input_4)
                            df[national_person_series] = df[national_person_series].mask(df[person_name_series]==j, input_4)
                        if input_3== 'nothing' or 'دخفاهدل':
                            continue
            c= c+1
        end_time= datetime.datetime.now()
        print('Duration: {}'.format(end_time - start_time))
    print('there is no problem in word dictation or code')
    return df

In [ ]:
def path_length_histogram(G, title=None):
    # find path lengths
    length_source_target= dict(nx.shortest_path_length(G))
    # convert dict of dicts to flat list
    all_shortest = sum([ 
        list(length_target.values())
        for length_target
        in length_source_target.value()],
    [])
    # calculate integer bins
    high= max(all_shortest)
    bins= [-0.5 + i for i in range (high + 2)]
    # plot histogram
    plt.hist(all_shortest, bins=bins, rwidth=3)
    plt.title(title)
    plt.xlable('Distance')
    plt.ylabel('Count')

In [ ]:
def nodes_connected(G, u, v):
    return u in G.neighbors(v)

In [ ]:
def finding_layers_node (G, start, k):
    nbrs= list([start])
    for l in range(k):
        nbrs= list(nbr for n in nbrs for nbr in G[n])
    return nbrs

In [ ]:
def insert_code(df, code_column_name, name_column_name):
    def checking_dictation(df, person_name):
        input_1 = input(f'insert new dictation for {person_name}: ')
        list_index_name= df.loc[df['person_name']== person_name].index_tolist()
        df.loc[list_index_name , ['person_name']] == [input_1]
    nan_person_national_code= df.loc[df[code_column_name]=='nan']
    nan_person_name= list(nan_person_national_code[name_column_name])
    nan_person_name_drop_duplicate= list(dict.fromkeys(nan_person_name))
    nan_person_name_drop_duplicate.sort(reversed=True)
    print('len nan_person_name ', len(nan_person_name))
    print('len nan_person_name_drop_duplicate ', len(nan_person_name_drop_duplicate))
    for i in nan_person_name_drop_duplicate:
        list_index= df.loc[df[name_column_name]==i].index.tolist()
        input_1= input(f'for {i} : (code/dict/next/done)')
        if input_1=='code'or input_1=='زخیث':
            input_3= input(f'insert code for {i} :')
            df.loc[list_index, [code_column_name]]=[input_3]
            input_2= input('Running checking dictation (n/y)')
            if input_2=='y' or input_2=='غ':
                checking_dictation(df, person_name=i)
            if input_2=='n' or input_2=='د':
                continue
        if input_1=='dict' or 'یهزف':
            input_3 = input(f'new dict for {i} :')
            df.loc[list_index, [name_column_name]]=[input_3]
        if input_1== 'next' or 'دثطف':
            continue
        if input_1== 'done' or input_1 =='یخدث':
            break
    return df

# Default measuers

In [ ]:
%matplotlib inline
plt.rcParams.update({ 
    'figure.figsize' : (24 , 24),
    'axes.spines.right' : False,
    'axes.spines.left' : False,
    'axes.spines.top' : False,
    'axes.spines.bottom' : False
})

In [ ]:
seed= hash('Network Science in python') %2**32
nprnd.seed(seed=seed)
random.seed(seed)

In [ ]:
today= datetime.date.today()

# Server connection

In [ ]:
server= 'server name'
database= 'database name'
username= 'username'
password= 'your username password'

In [ ]:
cnxn = db.connect( 
    'DRIVER={SQL Server}; SERVER=' +server+';DATABASE=' +database+';UTD'+username+';PWD'+password
)
cursor= cnxn.cursor()

# Data

In [ ]:
file_adress= input(r'insert adress of raw file: ')
df= read_merge_output(path=f'{file_adress}')

In [ ]:
change_column_name(df=df)

In [ ]:
code_columns= column_clustering(df)[1]

In [ ]:
string_columns= column_clustering(df)[0]

In [ ]:
df_copy= df.copy()
df_copy.info()

In [ ]:
d_copy= code_clean(df_copy, code_columns)
df_copy= string_clean(df_copy, string_columns)

In [ ]:
bank_file= input('bank name: ')
df_copy.to_csv(f'saveing adress', index=False, encoding='utf-8-sig')

In [ ]:
for index, row in df_copy.iterrows():
    sursor.execute('''
                    INSERT INTO table name (column names)
                    VALUES (?)
                    ''',
                    row.loc['name_column']
                    )
cbxb.commit()

# cleansing data

## arange data

In [ ]:
agent_index= df_copy.loc[df_copy['agent_name'] != 'nan'].index.tolist()
df_index= df_copy.index.tolist()
rest_index= list(set(df_index)-set(agent_index))

In [ ]:
print(len(agent_index))
print(len(df_index))
print(len(rest_index))

In [ ]:
df_agent['agent_name_1']= df_agent['person_name']
df_agent['agent_national_code_1']= df_agent['person_national_code']
df_agent['agent_id_1']= df_agent['person_id']
df_agent= df+agent.drop(columns=['person_name','person_national_code','person_id'])
df_agent['person_name']= df_agent['agent_name']
df_agent['person_id']= df_agent['agent_id']
df_agent['person_national_code']= df_agent['agent_national_code']
df_agent= df_agent.drop(columns=['agent_name','agent_id','agent_national_code'])
df_agent= df_agent.rename(columns={'agent_name_1':'agent_name','agent_national_code_1':'agent_national_code','agent_id_1':'agent_id'})
df_agent= df_agent[['company_id','company_national_code','company_name','date','charge_id','charge','person_name','person_id','person_national_code','agent_id','agent_national_code','agent_name','case_national_code']]
df_agent['person_type']=2
print(f'df_agent : {list(df_agent.columns)} , {len(list(df_agent.columns))}')
df_agent

In [ ]:
df_grapg= df_copy.loc[rest_index]
print(f' df_graph: {list(df_graph.columns)}, {len(list(df_graph.columns))}')

In [ ]:
df_graph= df_graph.append(df_agent).resrt_index().drop(columns=['index'])
print(f'df_graph: {list(df_graph.columns)} , {len(df_graph.columns)}')

In [ ]:
len_graph_df= len(df_graph)
len_graph_df

In [ ]:
if len(df_graph) == len(df_copy):
    print('program complete')
else:
    print('there is problem')

In [ ]:
df_graph.info()

## making timestamp

In [ ]:
df_graph['date']= df_graph['date'].apply(lambda x: x.replace('/',''))
df_graph['date_timestamp']= date_corrector(df_graph['date'])
df_graph['year']= df_graph['date_timestamp'].apply(lambda x: x.split('-')[0])

In [ ]:
df_graph.info()

## person name problem

### person name with person national code

In [ ]:
df_graph= checking_words(df=df_graph)

In [ ]:
if len(dg_graph) == len_graph_df:
    print('program complete')
else:
    print('there is problem')

### person name without person national code

In [ ]:
nan_index= df_graph.loc[df_graph.person_national_code=='nan']
nan_index= nan_index.loc[nan_index.person_type==2].index.tolist()
graph_index= df_graph.index.tolist()
rest_index= list(set(graph_index)-set(nan_index))

In [ ]:
print(f'length of data without person national code: {len(nan_index)}')
print(f'length of data with person national code: {len(rest_index)}')
print(f'length of main data: {len(graph_index)}')

In [ ]:
df_graph_nan= df_graph.loc[nan_index]
df_graph_nan= insert_code(df_graph_nan, 'person_national_code','person_name')

In [ ]:
df_graph_non_nan= df_graph.loc[rest_index]
df_graph= df_graph_non_nan.append(df_graph_nan).reset_index().drop(colmns=['index'])
print(f'df_graph: {list(df_graph.column)} \nnumber of column is: {len(list(df_graph.column))} \nlength df_graph is: {len(df_graph)}')

In [ ]:
df_graph= checking_words(df=df_graph)

some person name that doesnt have national code is remain
for this group should set random (rang) national number

In [ ]:
df_without_id= df_graph.loc[df_graph['person_national_code']=='nan']
person_without_id= list(dict.fromkeys(df_without_id['person_name']))
len_without_id= len(person_without_id)

In [ ]:
id_range= list(range(1, lem_without_id + 1))
len(id_range)

In [ ]:
c= 0
for i in person_without_id:
    df_graph['person_national_code']= df_graph['person_national_code'].nask(df_graph['person_name']==i, str(id_range[0]))
    c = c + 1

### checking company name with person name

In [ ]:
checking_company_person(df=df_graph, national_company_series='comapny_national_code', national_person_series='person_national_code',company_name_series='company_name',person_name_series='persona_name')

## final data

In [ ]:
df_graph.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

In [ ]:
for index, row in df_graph.itterwos():
    cursor.execute('''
                INSERT INTO table_name (columns name)
                VALUES (?)
                ''',
                row.loc['column name']
                )
cnxn.commit()

# network graph

## choose year

In [ ]:
selected_year= int(input(f'select year from{list(dict.fromkeys(df_graph.year))}'))

In [ ]:
df_graph_year= df_graph.loc[df_graph.year > selected_year].reset_index().drop(columns='index')

In [ ]:
list(dict.fromkeys(df_graph_year.year))

## making edge list

In [ ]:
df_graph_year['tuple_edge']= list(df_graph_year[['person_national_code','company_national_code']].itertuple(index=False, name=None))
edge_list= list(df_graph_year['tuple_edge'])

## making node dataframe and list

In [ ]:
df_source= df_graph_year[['person_name','person_national_code','person_type']]
df_target= df_graph_year[['company_name','company_national_code']]
df_source.rename(columns={'person_name':'name','person_national_code':'national_coed','person_type':'type'}, inplace=True)
df_target.rename(columns={'company_name':'name','company_national_code':'national_code'}, inplace=True)
df_target['type']=2
df_node= df_source.append(df_target).reset_index().drop(columns=['index'])
df_node= df_node.reset_index()
df_node= df_node.drop_duplicates('national_code').reset_index().drop(columns=['level_0','index'])

In [ ]:
node_list= list(dict.fromkeys(df_node.national_code))
len(node_list)

## making graph

In [ ]:
G_year= nx.DiGraph()
G_year.add_nodes_from(node_list)
G_year.add_edges_from(edge_list)
directed_pos= nx.spring_layout(G_year, k=1.5)

In [ ]:
print(f'len edge: {len(G_year.edges())}')
print(f'len node: {len(G_year.nodes())}')

## adding attributes

### node attributes

In [ ]:
node_name= list(df_node.name)
node_type= list(df_node.type)
print(f'length of node name" {len(node_name)}')
print(f'length of node type: {len(node_type)}')

In [ ]:
for node_id in G_year.nodes:
    node_index= df_node.loc[df_node['national_code']==node_id].index.tolist()
    node_index= node_index[0]
    G_year.nodes[node_id]['name']=node_name[node_index]

In [ ]:
for node_id in G_year.nodes:
    node_index= df_node.loc[df_node['national_code']==node_id].index.tolist()
    node_index= node_index[0]
    G_year.nodes[node_id]['typr']=node_type[node_index]

In [ ]:
node_color=[ 
    '#bb7a44' if G_year.nodes[v]['type']==2
    else '#33a02c' for v in G_year
]

### edge attributes

In [ ]:
for v, w in G_year.edges:
    if G_year.nodes[v]['type']== G_year.nodes[w]['type']:
        G_year.edges[v, w]['company_company']= True
    else:
        G_year.edges[v, w]['company_company']= False

In [ ]:
co_co= [e for e in G_year.edges if G_year.edges[e]['company_company']]
co_per= [e for e in G_year.edges if ~G_year.edges[e]['company_company']]

## subgraph by attribute

making subgraph of the nodes that are company

the remain work on graph will be done on the subgraph of company nodes

In [ ]:
nodes_co= ( 
    node
    for node, data
    in G_year.nodes(data=True)
    if data.get('type')==2
)

In [ ]:
G_co= G_year.subgraph(nodes_co)

In [ ]:
print(f' length of G_co graph: {len(G_co.nodes())}')

In [ ]:
each_nodes= []
remove_nodes= []
for nodes in G_co.nodes():
    if G_co.degree(nodes)==1:
        each_nodes.append(nodes)
for i in each_nodes:
    for j in list(G_co.out_edge(i)):
        if G_co.degree(j[1])==1:
            remove_nodes(i)

In [ ]:
G= nx.Digraph(G_co)
G.remove_nodes_from(remove_nodes)

## subgraph related to specific node

In [ ]:
specific_node_code= int(input('insert specific node national code: '))
node_related= list(nx.descendants(G, specific_node_code))
node_related.append(specific_node_code)
node_related= list(dict.fromkeys(node_related))
G_node_related= G.subgraph(node_related)

In [ ]:
G_node_related_edge = list(G_node_related.edges())

In [ ]:
G_node_related_df= pd.DataFrame(G_node_related_egde)
G_node_related_df.rename(column={0:'source',1:'target'}, inplace=True)
G_node_related_df['bank_name']= 'pasargad'
G_node_related_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

In [ ]:
for index, row in G_node_related_df.iterrow():
    cursor.execute('''
                INSERT INTO table_name (columns name)
                VALUES (?)
                ''',
                row.loc['columns name']
                )
cnsn.commit()

In [ ]:
node_related = nx.get_node_attributes(G_node_related, name='name')
node_related_df= pd.DataFrame(list(name.items()))
node_related_df['bank_name']='pasargad'
node_related_df.rename(columns={0:'id',1:'label'}, inplace=True)
node_related_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

In [ ]:
for index, row in node_related_df:
    cursor.execute('''
                ISERT INTO table_name (columns)
                VALUES (?)
                ''',
                row.loc['column name']
                )
cnxn.commit()

## find complement graph

In [ ]:
G_C= nx.complement(G_node_related)
nx.is_strongly_connected(G_C)
G_C_edge= list(G_C.edges())
G_C_df= pd.DataFrame(G_C_edge)
G_C_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

## finding connected nodes by layers

### first method

find connected nodes to specific node during given layer

In [ ]:
connected_node_national_code= int(input('onsert national_code for finding connected nodes'))
layer_number= int(input('number of layers to find'))
tereshold= list(nx.single_source_shortest_path_length(G_node_related, connected_node_national_code, cutoff= layer_number))

In [ ]:
list_nodes_tereshold= []
for i in tereshold:
    name= G_node_related.nodes[i]
    list_nodes_tereshold.append(name)

In [ ]:
list_nodes_tereshold

### secind method

find connected nodes to specific node by each layer

In [ ]:
connected_node_national_code= int(input('onsert national_code for finding connected nodes'))
layer_number= int(input('number of layers to find'))
if __name__ == '__main__':
    print(finding_layers_node(G_node_related, connected_node_national_code,layer_number))

## find strongly connected components

In [ ]:
str_con= list(nx.strongly_connected_components(G_node_related))
list_name=[]
c= 0
for item in str_con:
    if len (item) > 1:
        list_n=[]
        for j in item:
            name= G_bank.nodes[j]['name']
            list_n.append(name)
        list_name.append(list_n)
        G_layer = G_node_related.subgraph(item)
        G_layer_edge= list(G_layer.edges())
        G_layer_df= pd.DataFrame(G_layer_edge)
        G_layer_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')
        c = c + 1

# network analysis

## calculate indexes for specific subgraph

### betweenness

In [ ]:
betweenness= nx.betweenness_centrality(G_node_related, normalized=False, endpoints=True)
betweenness_sorted= sorted(betweenness.items(), key= lambda x: x[1], reverse= True)[0:10]
list_betweenness=[]
for i in betweennes_sorted:
    print(G_node_related.nodes[i[0]])
    list_betweenness.append(G_node_related.nodes[i[0]])

In [ ]:
betweenness_df= pd.DataFrame(list_betweenness)
betweenness_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### edge betweenness

use this index to finding importatnt edge in graph

In [ ]:
edge_betweenness= nx.edge_betweenness_centrality(G_node_related, normalized=False, endpoints=True)
edge_betweenness_sorted= sorted(edge_betweenness.items(), key= lambda x: x[1], reverse= True)[0:10]
edge_betweenness_sorted

In [ ]:
important_edge_betweenness=[ids for (ids, measurement) in edge_betweenness_sorted]
important_edge_betweenness_list=[]
for i in important_edge_betweenness:
    name_1= G_node_related[i[0]]['name']
    name_2= G_node_related[i[1]]['name']
    tuple_name=(name_1 , name_2)
    important_edge_betweenness_list.append(tuple_name)
important_edge_betweenness_list

In [ ]:
edge_betweenness_df= pd.DataFrame(important_edge_betweenness_list)
edge_betweenness_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### closeness

In [ ]:
closeness= nx.closeness_centrality(G_node_related, normalized=False, endpoints=True)
closeness_sorted= sorted(closeness.items(), key= lambda x: x[1], reverse= True)[0:10]
list_closeness=[]
for i in closeness_sorted:
    print(G_node_related.nodes[i[0]])
    list_closeness.append(G_node_related.nodes[i[0]])

In [ ]:
closeness_df= pd.DataFrame(list_closeness)
closeness_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### degree

In [ ]:
degree= G_node_related.degree()
degree_sorted= sorted(degree, key= lambda x: x[1], reverse= True)[0:10]
list_degree=[]
for i in degree_sorted:
    print(G_node_related.nodes[i[0]])
    list_degree.append(G_node_related.nodes[i[0]])

In [ ]:
degree_df= pd.DataFrame(list_degree)
degree_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### degree centrality

In [ ]:
degree_centrality= (nx.degree_centrality(G_node_related))
degree_centrality_sorted= {k: v for k,v in sorted(degree_centrality.items(), key= lambda item: item[1], reverse=True)[:10]}
list_degree_centrality=[]
for i in degree_centrality_sorted:
    print(G_node_related.nodes[i])
    list_degree_centrality.append(G_node_related.nodes[i])

In [ ]:
degree_centrality_df= pd.DataFrame(list_degree_centrality)
degree_centrality_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### out degree

In [ ]:
out_degree= G_node_related.out_degree()
out_degree_sorted= sorted(out_degree, key= lambda x: x[1], reverse= True)[0:10]
list_out_degree=[]
for i in out_degree_sorted:
    print(G_node_related.nodes[i[0]])
    list_out_degree.append(G_node_related.nodes[i[0]])

In [ ]:
out_degree_df= pd.DataFrame(list_out_degree)
out_degree_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### out degree centrality

In [ ]:
out_degree_centrality= (nx.out_degree_centrality(G_node_related))
out_degree_centrality_sorted= {k: v for k,v in sorted(out_degree_centrality.items(), key= lambda item: item[1], reverse=True)[:10]}
list_out_degree_centrality=[]
for i in out_degree_centrality_sorted:
    print(G_node_related.nodes[i])
    list_out_degree_centrality.append(G_node_related.nodes[i])

In [ ]:
out_degree_centrality_df= pd.DataFrame(list_out_degree_centrality)
out_degree_centrality_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### in degree

in_degree= G_node_related.in_degree()
in_degree_sorted= sorted(in_degree, key= lambda x: x[1], reverse= True)[0:10]
list_in_degree=[]
for i in in_degree_sorted:
    print(G_node_related.nodes[i[0]])
    list_in_degree.append(G_node_related.nodes[i[0]])

In [ ]:
in_degree_df= pd.DataFrame(list_in_degree)
in_degree_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### in degree centrality

In [ ]:
in_degree_centrality= (nx.in_degree_centrality(G_node_related))
in_degree_centrality_sorted= {k: v for k,v in sorted(in_degree_centrality.items(), key= lambda item: item[1], reverse=True)[:10]}
list_in_degree_centrality=[]
for i in in_degree_centrality_sorted:
    print(G_node_related.nodes[i])
    list_in_degree_centrality.append(G_node_related.nodes[i])

In [ ]:
in_degree_centrality_df= pd.DataFrame(list_in_degree_centrality)
in_degree_centrality_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### katz centrality

computes the relative influence of a node within a network by measuring the number of the immediate neighbors and also all other nodes in the network that connect to the node under consideration through these immediate neighbors

In [ ]:
katz_centrality= nx.katz_centrality(G_node_related)
katz_centrality_sorted= {k: v for k,v in sorted(katz_centrality_sorted.items(), key= lambda item: item[1], reverse=True)[:10]}
list_katz_centrality=[]
for i in katz_centrality_sorted:
    print(G_node_related.nodes[i])
    list_katz_centrality.append(G_node_related.nodes[i])

In [ ]:
katz_centrality_df= pd.DataFrame(list_katz_centrality)
katz_centrality_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### load centrality

the load centrality of a node is the fraction of all shortest paths that pass through that node

In [ ]:
load_centrality= nx.load_centrality(G_node_related)
load_centrality_sorted= {k: v for k,v in sorted(load_centrality.items(), key= lambda item: item[1], reverse=True)[:10]}
list_load_centrality=[]
for i in load_centrality_sorted:
    print(G_node_related.nodes[i])
    list_load_centrality.append(G_node_related.nodes[i])

In [ ]:
load_centrality_df= pd.DataFrame(list_load_centrality)
load_centrality_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### harmonic centrality

is the sum of the reciprocal of the shortest path distances from all other nodes to the specific node

In [ ]:
harmonic_centrality= nx.harmonic_centrality(G_node_related)
harmonic_centrality_sorted= {k: v for k,v in sorted(harmonic_centrality.items(), key= lambda item: item[1], reverse=True)[:10]}
list_harmonic_centrality=[]
for i in harmonic_centrality_sorted:
    print(G_node_related.nodes[i])
    list_harmonic_centrality.append(G_node_related.nodes[i])

In [ ]:
harmonic_centrality_df= pd.DataFrame(list_harmonic_centrality)
harmonic_centrality_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### local reaching centrality

the local reaching centrality of a node in a directed graph is the proportion of other nodes reachable from that node

In [ ]:
selected_national_code= int(input('insert national code for calculate local reaching centrality for: '))
nx.local_reaching_centrality(G_node_related, selected_national_code)

### global reaching centrality

the global reaching centrality of weighted directed graph is the average over all nodes of the difference between the local reaching centrality of the node and the gratest local reaching centrality of any node in the graph

In [ ]:
nx.global_reaching_centrality(G_node_related)

### average neighbor degree

In [ ]:
average_neighbor_degree= nx.average_neighbor_degree(G_node_related)
average_neighbor_degree_sorted= {k: v for k,v in sorted(average_neighbor_degree.items(), key= lambda item: item[1], reverse=True)[:10]}
list_average_neighbor_degree=[]
for i in average_neighbor_degree_sorted:
    print(G_node_related.nodes[i])
    list_average_neighbor_degree.append(G_node_related.nodes[i])

In [ ]:
average_neighbor_degree_df= pd.DataFrame(list_average_neighbor_degree)
average_neighbor_degree_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### trophic levels

In [ ]:
trophic_levels= nx.trophic_levels(G_node_related)
trophic_levelssorted= {k: v for k,v in sorted(trophic_levels.items(), key= lambda item: item[1], reverse=True)[:10]}
list_trophic_levels=[]
for i in trophic_levels_sorted:
    print(G_node_related.nodes[i])
    list_trophic_levels.append(G_node_related.nodes[i])

In [ ]:
trophic_levels_df= pd.DataFrame(list_trophic_levels)
trophic_levels_df.to_csv('saveing file adress', index=False, encoding='utf-8-sig')

### trophic differences

In [ ]:
nx.trophic_differences(G_node_related)

### trophic incoherence parameter

In [ ]:
nx.trophic_incoherence_parameter(G_node_related)

### voterank

computes a ranking of the nodes in a graph based on a voting scheme. with voterank all nodes vote for each of its in neighbours and the node with the highest wotes is selected iteratively. the voting ability of out neighbours of selected nodes is decreased in subsquent turns

In [ ]:
voterank_list= nx.voterank(G_node_related)
vote_rank_name=[]
for i in voterank_list:
    name= G_node_related.nodes[i]['name']
    vote_rank_name.append(name)
vote_rank_name

### other indexes

#### eccentricity

In [ ]:
print('Eccentricity: ', nx.eccentricity(G_node_related))

#### diameter

In [ ]:
print('Diameter: ', nx.diameter(G_node_related))

#### radius

In [ ]:
print('Radius: ', nx.radius(G_node_related))

#### preiphery

In [ ]:
print('Preiphery: ', nx.periphery(G_node_related))

#### center

In [ ]:
print('Center: ', nx.center(G_node_related))